<a href="https://colab.research.google.com/github/tirabo/Algoritmos-y-Programacion/blob/main/Objetos_1_2021_05_19_Algoritmos_y_Programaci%C3%B3n_tipos_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tipos abstractos de datos

Hemos visto ejemplos y ejercicios con muchas de las estructuras de datos usuales de los lenguajes de programación: enteros, reales, booleanos, strings, arreglos, tuplas, listas.

Con estas estructuras de datos también es posible representar otras. Por ejemplo, hemos representado los naturales usando enteros y hemos representado las fechas utilizando ternas de enteros.

De todas formas, no se obtiene el mismo grado de confianza en el funcionamiento de las operaciones sobre los tipos de datos nuestros que sobre los de Python: cuando representamos, por ejemplo, a las fechas como ternas de enteros, no hay ninguna garantía de que el programador los utilice apropiadamente: por error puede alterar el valor de uno de los elementos de la terna y transformar una fecha válida en una que no lo es.

De la misma manera, cuando representamos a los naturales como enteros, puede ocurrir que por error modificamos el número y pasa a ser negativo.

En nuestro auxilio surge la posibilidad que tienen la mayoría de los lenguajes de programación de definir apropiadamente un tipo nuevo, de manera de que el funcionamiento sea más confiable, entre muchas otras ventajas. La idea es definir un tipo nuevo y sus operaciones, de manera de que luego solamente se lo pueda manipular a través de dichas operaciones.

En Python, definir un tipo nuevo es definir una clase.

In [ ]:
def es_bisiesto(anho: int) -> bool:
    # pre: anho > 0
    # post: devuelve True si anho  es divisible por 4 (NO corresponde siempre a un año bisiesto)
    precondición = type(anho) == int and anho > 0
    assert precondición, 'Error: anho debe ser un entero positivo.'

    return anho % 4 == 0 and anho % 100 != 0 or anho % 400 == 0

def es_fecha_valida(fecha: tuple) -> bool:
    # pre: fecha es una terna de enteros positivos
    # post: devuelve True sii la terna se corresponde con el día, mes y año de una fecha válida
    precondición = type(fecha) == tuple and all(type(x) == int and x > 0 for x in fecha)
    assert precondición, 'Error: fecha debe ser una terna de enteros positivos.'

    dia, mes, anho = fecha

    anho_ok = 1 <= anho
    mes_ok = 1 <= mes <= 12
    if mes in [4, 6, 9, 11]:
        dia_ok = 1 <= dia <= 30
    elif mes == 2:
        dia_ok = 1 <= dia <= 28 or (dia == 29 and es_bisiesto(anho))
    else:
        dia_ok = 1 <= dia <= 31
    
    return dia_ok and mes_ok and anho_ok

def bisiestos_hasta(anho: int) -> int:
    # pre: anho es un año válido
    # post: devuelve el número de años bisiestos pasados incluyendo anho, si es bisiesto
    precondición = type(anho) == int and anho > 0
    assert precondición, 'Error: anho debe ser un entero positivo.'

    bisiestos_anteriores = anho // 4 - anho // 100 + anho // 400
    return bisiestos_anteriores

def dias_del_anho_actual(fecha: tuple) -> int:
    # pre: fecha es una fecha válida
    # post: devuelve el número de días transcurridos en el corriente año, contando el actual
    precondición = es_fecha_valida(fecha)
    assert precondición, 'Error: fecha debe ser una fecha válida.'

    # [días antes de enero, días antes de febrero, etc.] de un año no bisiesto
    DIAS_MESES_ANTERIORES = [0, 31, 59, 90, 120, 151, 181, 212, 243, 273, 304, 334]
    dia, mes, anho = fecha

    dias = dia + DIAS_MESES_ANTERIORES[mes-1]
    # si el año es bisiesto y la fecha es después de febrero, pasó un día más
    if es_bisiesto(anho) and mes > 2:
        dias = dias + 1
    return dias

def dias_de_la_era(fecha: tuple) -> int:
    # pre: fecha es una fecha válida
    # post: devuelve el número de días desde el 1/1/1 (asumiendo siempre Gregoriano) hasta la fecha, contando ambos extremos
    precondición = es_fecha_valida(fecha)
    assert precondición, 'Error: fecha debe ser una fecha válida.'

    dia, mes, anho = fecha

    nro_de_dias = (anho - 1) * 365 + bisiestos_hasta(anho-1) + dias_del_anho_actual(fecha)
    return nro_de_dias

def dia_de_la_semana(fecha: tuple) -> str:
    # pre: fecha es una fecha válida
    # post: devuelve el día de la semana en que cae la fecha dada
    precondición = es_fecha_valida(fecha)
    assert precondición, 'Error: fecha debe ser una fecha válida.'

    DIAS_DE_LA_SEMANA = ['domingo', 'lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado']
    return DIAS_DE_LA_SEMANA[dias_de_la_era(fecha) % 7]

def ingresar_fecha() -> tuple:
    # post: devuelve la primera terna de enteros positivos ingresada con formato día/mes/año

    esperando_fecha = True
    while esperando_fecha:
        cadena_ingresada = input('Ingrese fecha con formato día/mes/año: ')
        cadena_sin_espacios = ''.join(cadena_ingresada.split())
        cadenas_fecha = cadena_sin_espacios.split('/')
        if len(cadenas_fecha) == 3 and all(cadena.isdecimal() for cadena in cadenas_fecha):
            terna = [int(cadena) for cadena in cadenas_fecha]
            esperando_fecha = False
        else:
            print('Error: debe ingresar una fecha con formato día/mes/año, donde día, mes y año deben ser números enteros positivos. Inténtelo nuevamente.')
    return terna

def ingresar_fecha_valida() -> tuple:
    fecha = ingresar_fecha()
    while not es_fecha_valida(fecha):
        print('Error: debe ingresar una fecha válida. Inténtelo nuevamente.')
        fecha = ingresar_fecha()
    return fecha


Comencemos con la definición de la clase. Definir una **clase**, es similar a definir un **tipo**. Estamos diciendo qué características tendrán todos los **elementos** de dicho **tipo**.

En términos de lenguajes orientados a objetos, estamos diciendo qué características tendrán todos los **objetos** de dicha **clase**.

A cada objeto de la clase se le llama **instancia** de la clase.

Los ingredientes más importantes de la definición de una clase son

*   sus **atributos**, también llamados **campos** (fields) o **variables de instancias** (instance variables)
*   sus **métodos**, también llamados **métodos de instancias** (instance methods).

Los primeros son los datos que utilizamos para representar el estado de un objeto. Los segundos son las funciones que se utilizan para acceder a los datos y modificarlos.

In [ ]:
class Fecha:                                              # nombre de la clase
    def __init__(self, dma : tuple):                      # método inicializador
        self.terna = dma                                  # terna es un atributo

In [ ]:
t = (19, 5, 2021)
fecha = Fecha(t)                                          # Fecha() es el constructor
print(fecha.terna)
print(fecha)

Notemos que el método `__init__()` se invoca implícitamente cuando escribimos `Fecha()`.

Spoiler: una de las características que deseamos es que no se conozca la representación de las fechas.


In [ ]:
class Fecha:
    def __init__(self, dma):
        self.__terna = dma

In [ ]:
fecha = Fecha((19,5,2021))
print(fecha.__terna)

Parece muy inútil un tipo nuevo del que no podemos obtener ninguna información:

In [ ]:
class Fecha:
    def __init__(self, dma):
        self.__terna = dma
    def terna(self):
        return self.__terna

In [ ]:
fecha = Fecha((19,5,2021))
print(fecha.terna())

Entonces, ¿para qué tanto lío?

De esta manera no se revela la representación interna. Puede haber otra representación interna, por el motivo que fuera. Pero siempre se debe definir adecuadamente el método `terna()`.

Por ejemplo, puedo querer representar con una terna en orden amd:

In [ ]:
class Fecha:
    def __init__(self, dma):
        self.__terna = (dma[2], dma[1], dma[0])
    def terna(self):
        return self.__terna

In [ ]:
fecha = Fecha((19,5,2021))
print(fecha.terna())

Claramente deberíamos volver a darlo vuelta para que el método `terna()` devuelva lo mismo independientemente de la representación.

In [ ]:
class Fecha:
    def __init__(self, dma):
        self.__terna = (dma[2], dma[1], dma[0])
    def terna(self):
        return (self.__terna[2], self.__terna[1], self.__terna[0])

In [ ]:
fecha = Fecha((19,5,2021))
print(fecha.terna())

Esto nos lleva a reconsiderar el nombre del método, sería mejor llamarle `dma()` y no `terna()`

In [ ]:
class Fecha:
    def __init__(self, x):
        self.__terna = x
    def dma(self):
        return self.__terna
    def amd(self):
        return (self.__terna[2], self.__terna[1], self.__terna[0])

In [ ]:
fecha = Fecha((19,5,2021))
print(fecha.dma())
print(fecha.amd())

Podríamos permitir que al crear una fecha, se la pueda omitir, adoptando un valor por defecto:

In [ ]:
class Fecha:
    def __init__(self, dma = (1,1,1)):
        self.__terna = dma
    def dma(self):
        return self.__terna
    def amd(self):
        return (self.__terna[2], self.__terna[1], self.__terna[0])

In [ ]:
fecha = Fecha((19,5,2021))
print(fecha.dma())
print(fecha.amd())
fecha = Fecha()
print(fecha.dma())
print(fecha.amd())

Aunque en realidad sería más interesante que como valor por defecto tome la fecha actual. Más adelante veremos cómo hacerlo.

También podríamos agregarle robustez a nuestra definición. Que cuando tengamos un objeto Fecha, sea una fecha válida:

In [ ]:
class Fecha:
    def __init__(self, dma):
        assert es_fecha_valida(dma), 'Error: intento de crear una fecha no válida.'
        self.__terna = dma
    def dma(self):
        return self.__terna
    def amd(self):
        return (self.__terna[2], self.__terna[1], self.__terna[0])

In [ ]:
fecha = Fecha((19,5,2021))
print(fecha.dma())
print(fecha.amd())

In [ ]:
fecha = Fecha((39,5,2021))
print(fecha.dma())
print(fecha.amd())

Incorporemos algunos métodos sencillos: para conocer el día, el mes, el año, el siglo:

In [ ]:
class Fecha:
    def __init__(self, dma):
        assert type(dma) == tuple and all(type(z) == int for z in dma) and es_fecha_valida(dma), 'Error: intento de crear una fecha no válida.'
        self.__terna = dma
    def dma(self):
        return self.__terna
    def amd(self):
        return (self.__terna[2], self.__terna[1], self.__terna[0])
    def dia(self):
        return self.__terna[0]
    def mes(self):
        return self.__terna[1]
    def anho(self):
        return self.__terna[2]
    def siglo(self):
        return self.anho() // 100 + 1

In [ ]:
fecha = Fecha([19,5,2021])
print(fecha.dma())
print(fecha.amd())
print(fecha.dia())
print(fecha.mes())
print(fecha.anho())
print(fecha.siglo())

Podríamos agregar funciones para ver el día como día de la semana, el mes en letras, etc.

A estos métodos se les suele llamar 'getters', permiten al programador acceder a datos, no necesariamente su representación.

Existen también 'setters', son métodos que se utilizan para modificar datos del objeto, por ejemplo, un atributo.

En este caso, el objeto que definimos es inmutable, por lo que no tenemos manera de modificar sus datos:

In [ ]:
class Fecha:
    def __init__(self, dma):
        assert es_fecha_valida(dma), 'Error: intento de crear una fecha no válida.'
        self.__terna = dma
    def dma(self):
        return self.__terna
    def amd(self):
        return (self.__terna[2], self.__terna[1], self.__terna[0])
    def dia(self):
        return self.__terna[0]
    def mes(self):
        return self.__terna[1]
    def anho(self):
        return self.__terna[2]
    def siglo(self):
        return self.__terna[2] // 100 + 1
    def establecer_anho(self, anho):
        assert type(anho) == int and anho > 0, 'Error: intento de establecer un anho no válido'
        self.__terna[2] = anho

In [ ]:
fecha = Fecha((19,5,2021))
fecha.establecer_anho(2000)
print(fecha.dma())
print(fecha.amd())
print(fecha.dia())
print(fecha.mes())
print(fecha.anho())
print(fecha.siglo())


Dos posibilidades:

1.   que el método establecer devuelva otro objeto de la clase Fecha
2.   definir la clase Fecha como una clase de objetos mutables


## Que devuelva otro objeto

In [ ]:
class Fecha:
    def __init__(self, dma):
        assert es_fecha_valida(dma), 'Error: intento de crear una fecha no válida.'
        self.__terna = dma
    def dma(self):
        return self.__terna
    def amd(self):
        return (self.__terna[2], self.__terna[1], self.__terna[0])
    def dia(self):
        return self.__terna[0]
    def mes(self):
        return self.__terna[1]
    def anho(self):
        return self.__terna[2]
    def siglo(self):
        return self.__terna[2] // 100 + 1
    def establecer_anho(self, anho):
        assert type(anho) == int and anho > 0, 'Error: intento de establecer un anho no válido'
        return Fecha((self.__terna[0], self.__terna[1], anho))

In [ ]:
fecha = Fecha((19,5,2021))
fecha2 = fecha.establecer_anho(2000)
print(fecha.dma())
print(fecha.amd())
print(fecha.dia())
print(fecha.mes())
print(fecha.anho())
print(fecha.siglo())
print(fecha2.dma())
print(fecha2.amd())
print(fecha2.dia())
print(fecha2.mes())
print(fecha2.anho())
print(fecha2.siglo())


## Que sea mutable

In [ ]:
class Fecha:
    def __init__(self, dma):
        assert es_fecha_valida(dma), 'Error: intento de crear una fecha no válida.'
        self.__terna = list(dma)
    def dma(self):
        return tuple(self.__terna)
    def amd(self):
        return (self.__terna[2], self.__terna[1], self.__terna[0])
    def dia(self):
        return self.__terna[0]
    def mes(self):
        return self.__terna[1]
    def anho(self):
        return self.__terna[2]
    def siglo(self):
        return self.__terna[2] // 100 + 1
    def establecer_anho(self, anho):
        assert type(anho) == int and anho > 0, 'Error: intento de establecer un anho no válido'
        self.__terna[2] = anho

In [ ]:
fecha = Fecha((19,5,2021))
fecha.establecer_anho(2000)
print(fecha.dma())
print(fecha.amd())
print(fecha.dia())
print(fecha.mes())
print(fecha.anho())
print(fecha.siglo())


¿Por qué escribió la lista en la primera línea? ¿Acaso estamos exhibiendo la representación interna?

In [ ]:
t = (19, 5, 2021)
print(t)
lt = list(t)
print(lt)

¿No puede la clase Fecha ser mutable a pesar de haberse elegido tuplas (que son inmutables) para la representacion?

In [ ]:
class Fecha:
    def __init__(self, dma):
        assert es_fecha_valida(dma), 'Error: intento de crear una fecha no válida.'
        self.__terna = dma
    def dma(self):
        return self.__terna
    def amd(self):
        return (self.__terna[2], self.__terna[1], self.__terna[0])
    def dia(self):
        return self.__terna[0]
    def mes(self):
        return self.__terna[1]
    def anho(self):
        return self.__terna[2]
    def siglo(self):
        return self.__terna[2] // 100 + 1
    def establecer_anho(self, anho):
        assert type(anho) == int and anho > 0, 'Error: intento de establecer un anho no válido'
        self.__terna = (self.__terna[0], self.__terna[1], anho)

In [ ]:
fecha = Fecha((19,5,2021))
fecha.establecer_anho(2000)
print(fecha.dma())
print(fecha.amd())
print(fecha.dia())
print(fecha.mes())
print(fecha.anho())
print(fecha.siglo())


Si aceptamos que un objeto pueda ser construido con una lista, debe tomarse el recaudo de generar una nueva lista como representación interna (en caso de optarse por lista como representación interna), para evitar que desde afuera del objeto pueda modificarse la representación interna al modificar la lista que se usó para construir el objeto.

In [ ]:
class Fecha:
    def __init__(self, dma):
        assert type(dma) == list and all(type(z) == int for z in dma), 'Error: el argumento debe ser una lista de enteros'
        self.__terna = dma
    def dma(self):
        return self.__terna
    def amd(self):
        return [self.__terna[2], self.__terna[1], self.__terna[0]]
    def dia(self):
        return self.__terna[0]
    def mes(self):
        return self.__terna[1]
    def anho(self):
        return self.__terna[2]
    def siglo(self):
        return self.__terna[2] // 100 + 1
    def establecer_anho(self, anho):
        assert type(anho) == int and anho > 0, 'Error: intento de establecer un anho no válido'
        self.__terna[2] = anho

In [ ]:
lista = [19, 5, 2021]
fecha = Fecha(lista)
print(fecha.dma())
print(fecha.amd())
lista[2] = 2000
print(fecha.dma())
print(fecha.amd())


Como vemos, al modificar la lista se modificó el objeto. Esto implica un acceso "escandaloso" a la representación interna: se la modificó sin apelar a ninguno de los métodos del objeto.

Se solucionando generando una copia de la lista en el constructor.

In [ ]:
class Fecha:
    def __init__(self, dma):
        assert type(dma) == list and all(type(z) == int for z in dma), 'Error: el argumento debe ser una lista de enteros'
        self.__terna = dma[:]
    def dma(self):
        return self.__terna
    def amd(self):
        return [self.__terna[2], self.__terna[1], self.__terna[0]]
    def dia(self):
        return self.__terna[0]
    def mes(self):
        return self.__terna[1]
    def anho(self):
        return self.__terna[2]
    def siglo(self):
        return self.__terna[2] // 100 + 1
    def establecer_anho(self, anho):
        assert type(anho) == int and anho > 0, 'Error: intento de establecer un anho no válido'
        self.__terna[2] = anho

In [ ]:
lista = [19, 5, 2021]
fecha = Fecha(lista)
print(fecha.dma())
print(fecha.amd())
lista[2] = 2000
print(fecha.dma())
print(fecha.amd())
fecha.establecer_anho(2000)
print(fecha.dma())
print(fecha.amd())
